import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=1092
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Haptics0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (155, 1093)
first_test_shape= (308, 1093)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [18 34 34 36 33]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -11.147
first_train_sample=
 [ 5.      -1.0475   0.54834 ...  0.89676 -2.6913  -8.1963 ]
1 18	2 34	3 34	4 36	5 33	

In [3]:
np.shape(ecg)

(155, 1093)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 18	2 34	3 34	4 36	5 33	
 max = 36


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (18, 1093)
Up to class  2 train shape =  (52, 1093)
Up to class  3 train shape =  (86, 1093)
Up to class  4 train shape =  (122, 1093)
Up to class  5 train shape =  (155, 1093)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 18 34 34 36 33]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  3.903199999999999
min magnitude class 1  =  -14.859999999999998
after normalizing >>
max magnitude class 1  =  1.1093335015171584
min magnitude class 1  =  -1.5203887849419413

 cls 2  >> 
max magnitude class 2  =  2.7426000000000004
min magnitude class 2  =  -10.213999999999999
after normalizing >>
max magnitude class 2  =  0.9466717121814145
min magnitude class 2  =  -0.8692370761242035

 cls 3  >> 
max magnitude class 3  =  2.9853
min magnitude class 3  =  -10.375999999999998
after normalizing >>
max magnitude class 3  =  0.9806868907716135
min magnitude class 3  =  -0.8919418924884895

 cls 4  >> 
max magnitude class 4  =  2.5669999999999997
min magnitude class 4  =  -9.867799999999999
after normalizing >>
max magnitude class 4  =  0.9220608124680276
min magnitude class 4  =  -0.8207160426345994

 cls 5  >> 
max magnitude class 5  =  2.8078999999999996
min magnitude class 5  =  -10.584999999999997
after normalizing >>
max magnitude class 5  = 

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.5203887849419413
max tst = 1.1093335015171584


In [14]:
len(wndws_test)

308

In [15]:
np.shape(wndws_test)

(308, 1093)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(18, 1093)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (155, 1092)
y_train =>  (155,)
X_test  =>  (308, 1092)
y_test  =>  (308,)
X_valid  =>  (155, 1092)
y_valid  =>  (155,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
8/8 [==============================] - 3s 92ms/step - loss: 1.8720 - accuracy: 0.1742 - val_loss: 1.6728 - val_accuracy: 0.2129
Epoch 2/1000
8/8 [==============================] - 0s 17ms/step - loss: 1.6769 - accuracy: 0.2065 - val_loss: 1.6186 - val_accuracy: 0.2774
Epoch 3/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6365 - accuracy: 0.2387 - val_loss: 1.6157 - val_accuracy: 0.2129
Epoch 4/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6460 - accuracy: 0.1742 - val_loss: 1.5902 - val_accuracy: 0.2129
Epoch 5/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6148 - accuracy: 0.1742 - val_loss: 1.5893 - val_accuracy: 0.2194
Epoch 6/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6002 - accuracy: 0.1742 - val_loss: 1.5834 - val_accuracy: 0.2129
Epoch 7/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6335 - accuracy: 0.1484 - val_loss: 1.6377 - val_accuracy: 0.2129
Epoch 8/1000


Epoch 59/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.7232 - accuracy: 0.7032 - val_loss: 0.4889 - val_accuracy: 0.8065
Epoch 60/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.6352 - accuracy: 0.7677 - val_loss: 0.7273 - val_accuracy: 0.6968
Epoch 61/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.5441 - accuracy: 0.7935 - val_loss: 0.5590 - val_accuracy: 0.7935
Epoch 62/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.5439 - accuracy: 0.7935 - val_loss: 0.5286 - val_accuracy: 0.7419
Epoch 63/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.6210 - accuracy: 0.7548 - val_loss: 0.4781 - val_accuracy: 0.7806
Epoch 64/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.4411 - accuracy: 0.8129 - val_loss: 0.4996 - val_accuracy: 0.7806
Epoch 65/1000
8/8 [==============================] - 0s 18ms/step - loss: 0.5371 - accuracy: 0.7871 - val_loss: 0.4170 - val_accuracy: 0.8387
Epoch 

 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 118, 128)          49280     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 39, 128)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 37, 256)           98560     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 12, 256)          0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 2000)              6146000   
          

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 3ms/step
array([[ 6,  8, 15, 10, 21],
       [ 4, 33,  5, 11,  5],
       [10,  9, 27,  7,  6],
       [ 1, 14,  5, 28, 16],
       [ 3, 12,  7,  9, 36]], dtype=int64)
              precision    recall  f1-score   support

           2       0.25      0.10      0.14        60
           3       0.43      0.57      0.49        58
           4       0.46      0.46      0.46        59
           5       0.43      0.44      0.43        64
           6       0.43      0.54      0.48        67

    accuracy                           0.42       308
   macro avg       0.40      0.42      0.40       308
weighted avg       0.40      0.42      0.40       308



In [27]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
8/8 [==============================] - 1s 57ms/step - loss: 1.9550 - accuracy: 0.1742 - val_loss: 1.6712 - val_accuracy: 0.2194
Epoch 2/1000
8/8 [==============================] - 0s 21ms/step - loss: 1.6700 - accuracy: 0.2000 - val_loss: 1.6107 - val_accuracy: 0.2194
Epoch 3/1000
8/8 [==============================] - 0s 26ms/step - loss: 1.6369 - accuracy: 0.1806 - val_loss: 1.6074 - val_accuracy: 0.2323
Epoch 4/1000
8/8 [==============================] - 0s 20ms/step - loss: 1.6454 - accuracy: 0.2000 - val_loss: 1.6118 - val_accuracy: 0.2194
Epoch 5/1000
8/8 [==============================] - 0s 20ms/step - loss: 1.6331 - accuracy: 0.2000 - val_loss: 1.6050 - val_accuracy: 0.2194
Epoch 6/1000
8/8 [==============================] - 0s 20ms/step - loss: 1.6198 - accuracy: 0.1871 - val_loss: 1.6013 - val_accuracy: 0.2194
Epoch 7/1000
8/8 [==============================] - 0s 20ms/step - loss: 1.6100 - accuracy: 0.1484 - val_loss: 1.6032 - val_accuracy: 0.2194
Epoch 8/1000


Epoch 59/1000
8/8 [==============================] - 2s 300ms/step - loss: 0.8763 - accuracy: 0.6323 - val_loss: 0.7495 - val_accuracy: 0.7419
Epoch 60/1000
8/8 [==============================] - 0s 23ms/step - loss: 0.8595 - accuracy: 0.6645 - val_loss: 0.6956 - val_accuracy: 0.7548
Epoch 61/1000
8/8 [==============================] - 0s 21ms/step - loss: 0.8119 - accuracy: 0.6774 - val_loss: 0.7489 - val_accuracy: 0.7097
Epoch 62/1000
8/8 [==============================] - 0s 22ms/step - loss: 0.7473 - accuracy: 0.7097 - val_loss: 0.7474 - val_accuracy: 0.7097
Epoch 63/1000
8/8 [==============================] - 0s 21ms/step - loss: 0.6643 - accuracy: 0.7355 - val_loss: 0.8058 - val_accuracy: 0.6581
Epoch 64/1000
8/8 [==============================] - 0s 20ms/step - loss: 0.8789 - accuracy: 0.6903 - val_loss: 0.7402 - val_accuracy: 0.7161
Epoch 65/1000
8/8 [==============================] - 2s 320ms/step - loss: 0.7644 - accuracy: 0.6968 - val_loss: 0.6036 - val_accuracy: 0.7871
Epoc

8/8 [==============================] - 0s 20ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0398 - val_accuracy: 0.9806
Epoch 117/1000
8/8 [==============================] - 0s 20ms/step - loss: 0.2846 - accuracy: 0.9097 - val_loss: 0.1439 - val_accuracy: 0.9290
Epoch 118/1000
8/8 [==============================] - 0s 20ms/step - loss: 0.2655 - accuracy: 0.9355 - val_loss: 0.1548 - val_accuracy: 0.9355
Epoch 119/1000
8/8 [==============================] - 0s 20ms/step - loss: 0.0947 - accuracy: 0.9806 - val_loss: 0.0161 - val_accuracy: 1.0000
Epoch 120/1000
8/8 [==============================] - 0s 20ms/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 121/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0384 - accuracy: 1.0000
np.shape(X_test)= (308, 1092, 1)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Co

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 4ms/step
array([[12,  6, 12, 13, 17],
       [ 3, 27,  5, 20,  3],
       [12,  9, 23, 12,  3],
       [ 4, 12,  5, 32, 11],
       [ 6,  9,  7, 15, 30]], dtype=int64)
              precision    recall  f1-score   support

           2       0.32      0.20      0.25        60
           3       0.43      0.47      0.45        58
           4       0.44      0.39      0.41        59
           5       0.35      0.50      0.41        64
           6       0.47      0.45      0.46        67

    accuracy                           0.40       308
   macro avg       0.40      0.40      0.40       308
weighted avg       0.40      0.40      0.40       308

